# Exercise 5: Variance reduction methods

In [381]:
import numpy as np
import math
import scipy.stats as st 

In [382]:
n = 100 # sample size
exact_solution = math.e - 1

## 1.
Estimate the integral $$\int_0^1 e^x dx$$ by simulation (the crude Monte Carlo estimator). Use e.g. an estimator based on 100 samples and present the result as the point estimator and a confidence interval.

In [384]:
def crude_MC(n):
    U = np.random.uniform(0,1,size=n) # 100 random samples of U(0,1)
    X = []

    for i in range(n):
        X_i = np.exp(U[i])
        X.append(X_i)
    return X

X = crude_MC(n)
mean = np.mean(X)
std = np.std(X, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=n-1, loc=mean, scale=std / np.sqrt(n))

print(f"Exact solution: {exact_solution}")
print(f"The crude Monte Carlo point estimator: {mean}")
print(f"The variance: {std**2}")
print(f"95% confidence interval: {ci}")

Exact solution: 1.718281828459045
The crude Monte Carlo point estimator: 1.6934037632538963
The variance: 0.2509606866355765
95% confidence interval: (1.5940024773838246, 1.792805049123968)


## 2.
Estimate the integral $$\int_0^1 e^x dx$$ using antithetic variables, with comparable computer ressources.

In [386]:
def antithetic(n):
    U = np.random.uniform(0,1,size=n) # 100 random samples of U(0,1)
    Y = []

    for i in range(n):
        Y_i = (np.exp(U[i]) + np.exp(1-U[i]))/2
        Y.append(Y_i)
    return Y

Y = antithetic(n)
mean = np.mean(Y)
std = np.std(Y, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=n-1, loc=mean, scale=std / np.sqrt(n))

print(f"Exact solution: {exact_solution}")
print(f"The Antithetic point estimator: {mean}")
print(f"The variance: {std**2}")
print(f"95% confidence interval: {ci}")

Exact solution: 1.718281828459045
The Antithetic point estimator: 1.7173343023652092
The variance: 0.004122805031632862
95% confidence interval: (1.7045938292933909, 1.7300747754370276)


## 3. 
Estimate the integral $$\int_0^1 e^x dx$$ using a control variable, with comparable computer ressources.

In [388]:
def control(n):
    U = np.random.uniform(0,1,size=n) # 100 random samples of U(0,1)
    Z = []
    
    X = np.exp(U)
    Y = U
    c = -np.cov(X,Y)[0, 1]/np.var(Y)
    for i in range(n):
        Z_i = X[i] + c*(U[i]-1/2) # X_i = exp(U_i)
        Z.append(Z_i)
    return Z

Z = control(n)
mean = np.mean(Z)
std = np.std(Z, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=n-1, loc=mean, scale=std / np.sqrt(n))

print(f"Exact solution: {exact_solution}")
print(f"The Control variable point estimator: {mean}")
print(f"The variance: {std**2}")
print(f"95% confidence interval: {ci}")

Exact solution: 1.718281828459045
The Control variable point estimator: 1.713973517500537
The variance: 0.003435586783460785
95% confidence interval: (1.7023432523173248, 1.7256037826837494)


## 4. 
Estimate the integral $$\int_0^1 e^x dx$$ using stratified sampling, with comparable computer ressources.

In [390]:
def stratified(n, m):
    W = []
    U = np.random.uniform(0,1,size=(n,m))
    for i in range(n):
        w_i = 0
        for j in range(m):
            w_i += np.exp(j/m+U[i,j]/m)
        W.append(w_i/m)
    return W

strata = 10
W = stratified(n,strata)
mean = np.mean(W)
std = np.std(W, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=n-1, loc=mean, scale=std / np.sqrt(n))

print(f"Exact solution: {exact_solution}")
print(f"The Control variable point estimator: {mean}")
print(f"The variance: {std**2}")
print(f"95% confidence interval: {ci}")

Exact solution: 1.718281828459045
The Control variable point estimator: 1.7212239480694496
The variance: 0.00031054285400649385
95% confidence interval: (1.7177273160871103, 1.7247205800517889)


## 5.
Use control variates to reduce the variance of the estimator in exercise 4 (Poisson arrivals).

In [392]:
# Known Parameters:
m = 10 # number of service units
mst = 8 # mean service time, in time units
mtbc = 1 # mean time between customers, in time units
batches = 10 # number of batches
customers_per_batch = 10_000 # number of customers per batch

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm
import scipy.stats as stats

# arrival times are poisson processes
# service times are exponential
def sim():
    at = np.cumsum(np.random.exponential(scale=mtbc, size=customers_per_batch)) # Poisson arrival times: CDF = P(Xi ≤ t) = 1 − e−λt
    st = np.random.exponential(scale=mst, size=customers_per_batch) # Exponential service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1

    return blocked_customers / customers_per_batch, np.mean(np.diff(at))

blocked_results = [sim () for batch in range(batches)]

X, Y = zip(*blocked_results)

# Control variate is the mean of the interarrival times which is 1/lambda = 1/mtbc = 1
control_blocked_frac = []
c = -np.cov(X, Y)[0, 1] / np.var(Y)
for i in range(batches):
    Z_i = X[i] + c * (Y[i] - 1 / mtbc)
    control_blocked_frac.append(Z_i)

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(control_blocked_frac)
std_blocked_fraction = np.std(control_blocked_frac, ddof=1)

# Calculate the 95% confidence interval
ci = stats.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction / np.sqrt(batches))

print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"Variance of fraction of blocked customers: {std_blocked_fraction**2}")
print(f"95% confidence interval: {ci}")


Mean fraction of blocked customers: 0.12060231843737668
Variance of fraction of blocked customers: 6.414020317816754e-06
95% confidence interval: (0.11879061153146984, 0.12241402534328351)


## 6.
Demonstrate the effect of using common random numbers in exercise 4 for the difference between Poisson arrivals (Part 1) and a renewal process with hyperexponential interarrival times. **Remark:** You might need to do some thinking and some re-programming.

**Solution:**

Let's apply inverse transform sampling to generate samples from an exponential distribution with rate parameter $\lambda$.
1. Exponential CDF

    For an exponential distribution with rate $\lambda$ :
    $$
    f_X(x)=\lambda e^{-\lambda x}, \quad x \geq 0
    $$
    
    The CDF is:
    $$
    F_X(x)=1-e^{-\lambda x}
    $$
2. Inverse CDF

    To find the inverse CDF, we set $F_X(x)=u$ and solve for $x$ :
    $$
    \begin{aligned}
    & u=1-e^{-\lambda x} \\
    & e^{-\lambda x}=1-u \\
    & -\lambda x=\ln (1-u) \\
    & x=-\frac{\ln (1-u)}{\lambda}
    \end{aligned}
    $$
    
    Since $U$ is uniformly distributed between 0 and $1,1-U$ is also uniformly distributed between 0 and 1. For simplicity, we can use $U$ directly:
    $$
    x=-\frac{\ln (U)}{\lambda}.
    $$

Now we can generate exponential and hyper-exponential inter-arrival times with a uniformly distributed variable $U$.

In [394]:
# Known Parameters:
m = 10 # number of service units
mst = 8 # mean service time, in time units
mtbc = 1 # mean time between customers, in time units
batches = 10 # number of batches
customers_per_batch = 10_000 # number of customers per batch
p1, lambda1, p2, lambda2 = 0.8, 0.8333, 0.2, 5.0 # Settings for the hyper-exponential distribution

def sim(at):
    st = np.random.exponential(scale=mst, size=customers_per_batch) # Exponential service times
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    for i in range(customers_per_batch):
        if any(service_busy_until <= at[i]):
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1

    return blocked_customers / customers_per_batch


### Not using a common random variable U ###
# Not using a common random number
at_exp = np.cumsum(np.random.exponential(scale=mtbc, size=customers_per_batch))

# Uniform variable only used to generate the hyper-exponential distribution
U_hyperexp = np.random.uniform(0, 1, size=customers_per_batch)

at_hyperexp = np.cumsum(np.where(
                        U_hyperexp < p1, 
                            np.random.exponential(scale=1/lambda1, size=customers_per_batch), 
                            np.random.exponential(scale=1/lambda2, size=customers_per_batch)
                        ))

# Exponential result
blocked_frac_poisson = [sim(at_exp) for batch in range(batches)]

# Hyper-exponential result with U
blocked_frac_hyperexponential = [sim(at_hyperexp) for batch in range(batches)]

diff = np.array(blocked_frac_poisson) - np.array(blocked_frac_hyperexponential)

# Calculate variance of difference between poisson and hyper-exponential results
variance_of_diff = np.std(diff)

### Using a common random variable U ###

# Generating uniform variables from 0 to 1, that are used in creating the two different arrival times
U = np.random.uniform(0, 1, size=customers_per_batch)

# Exponential inter-arrival times
at_exp_U = np.cumsum(-np.log(U)/mtbc)

# Hyper-exponential inter-arrival times
at_hyperexp_U = np.cumsum(np.where(
                        U < p1, 
                            np.random.exponential(scale=1/lambda1, size=customers_per_batch), 
                            np.random.exponential(scale=1/lambda2, size=customers_per_batch)
                        ))

# Exponential result with U
blocked_frac_poisson_U = [sim(at_exp_U) for batch in range(batches)]

# Hyper-exponential result with U
blocked_frac_hyperexponential_U = [sim(at_hyperexp_U) for batch in range(batches)]

diff_U = np.array(blocked_frac_poisson_U) - np.array(blocked_frac_hyperexponential_U)

# Calculate variance of difference between poisson and hyper-exponential results
variance_of_diff_U = np.std(diff_U)

print(f"Variance between the difference {variance_of_diff}")
print(f"Variance between the difference using common random variables U: {variance_of_diff_U}")

Variance between the difference 0.007495231817629124
Variance between the difference using common random variables U: 0.006383298520357635


## 7. 
For a standard normal random variable $Z \sim N(0, 1)$ using the crude Monte Carlo estimator, estimate the probability $Z > a$. Then try importance sampling with a normal density with mean $a$ and variance $\sigma^2$. For the experiments start using $\sigma^2$ = 1, use different values of $a$ (e.g. 2 and 4), and different sample sizes. If time permits experiment with other values for $\sigma^2$. Finally discuss the efficiency of the methods.

In [396]:
## Crude Monte Carlo ##

def crude_MC_probability(a, n):
    Z = np.random.normal(0,1,size=n)   
    return np.sum(Z > a)/n

ns = [1_000, 10_000, 100_000]
a_s = [2, 4]
var = 1
for n in ns:
    for a in a_s:
        p = crude_MC_probability(a, n)
        expected_p = 1 - stats.norm.cdf(a, 0, 1)
        print(f"Using a sample size of {n} and mean a of {a} we get:")
        print(f"Estimated probability that Z > {a} using crude Monte Carlo: {p}")
        print(f"Expected probability that Z > {a}: {expected_p}\n")



Using a sample size of 1000 and mean a of 2 we get:
Estimated probability that Z > 2 using crude Monte Carlo: 0.018
Expected probability that Z > 2: 0.02275013194817921

Using a sample size of 1000 and mean a of 4 we get:
Estimated probability that Z > 4 using crude Monte Carlo: 0.0
Expected probability that Z > 4: 3.167124183311998e-05

Using a sample size of 10000 and mean a of 2 we get:
Estimated probability that Z > 2 using crude Monte Carlo: 0.0236
Expected probability that Z > 2: 0.02275013194817921

Using a sample size of 10000 and mean a of 4 we get:
Estimated probability that Z > 4 using crude Monte Carlo: 0.0001
Expected probability that Z > 4: 3.167124183311998e-05

Using a sample size of 100000 and mean a of 2 we get:
Estimated probability that Z > 2 using crude Monte Carlo: 0.0234
Expected probability that Z > 2: 0.02275013194817921

Using a sample size of 100000 and mean a of 4 we get:
Estimated probability that Z > 4 using crude Monte Carlo: 6e-05
Expected probability th

For the importance sampling we notice that if we look at the structure of the importance sampling
$$\theta=\int \frac{h(x) f(x)}{g(x)} g(x) \mathrm{d} x=\mathrm{E}\left(\frac{h(Y) f(Y)}{g(Y)}\right)$$
that here we choose $f(x)$ as the PDF of a standard normal distribution, $g(x)$ as the PDF of a normal distribution with mean $a$ and variance $\sigma^2=1$ and $h(x)$ as the function that says if $Z>a$. We then also have $Y$ which is distributed with density $g(y)$.

In [398]:
## Importance sampling ##

def importance_sampling(a, sigma, n):
    Y = np.random.normal(a, sigma, n)
    
    f = stats.norm.pdf(Y)
    g = stats.norm.pdf(Y, a, sigma)
    h = np.where(Y > a, 1, 0)
    
    return np.mean(h*f/g)

ns = [1_000, 10_000, 100_000]
a_s = [2, 4]
sigma = 1
for n in ns:
    for a in a_s:
        p = importance_sampling(a, sigma, n)
        expected_p = 1 - stats.norm.cdf(a, 0, 1)
        print(f"Using a sample size of {n} and mean a of {a} we get:")
        print(f"Estimated probability that Z > {a} using crude Monte Carlo: {p}")
        print(f"Expected probability that Z > {a}: {expected_p}\n")



Using a sample size of 1000 and mean a of 2 we get:
Estimated probability that Z > 2 using crude Monte Carlo: 0.023554125629001534
Expected probability that Z > 2: 0.02275013194817921

Using a sample size of 1000 and mean a of 4 we get:
Estimated probability that Z > 4 using crude Monte Carlo: 3.3153919944585745e-05
Expected probability that Z > 4: 3.167124183311998e-05

Using a sample size of 10000 and mean a of 2 we get:
Estimated probability that Z > 2 using crude Monte Carlo: 0.022516105050181045
Expected probability that Z > 2: 0.02275013194817921

Using a sample size of 10000 and mean a of 4 we get:
Estimated probability that Z > 4 using crude Monte Carlo: 3.160212407140306e-05
Expected probability that Z > 4: 3.167124183311998e-05

Using a sample size of 100000 and mean a of 2 we get:
Estimated probability that Z > 2 using crude Monte Carlo: 0.022868409948656018
Expected probability that Z > 2: 0.02275013194817921

Using a sample size of 100000 and mean a of 4 we get:
Estimated 

**Discussion:** We see that the crude Monte Carlo method isn't as effective when we need samples for the extreme cases, as can be easily seen for $a=4$ and $n=1000$ since it outputs 0.0. Here is where the importance sampling comes in as it sort of gives greater weight to these extreme cases and we are able to get these cases more often which generates a better estimate, even with a small sample size.

## 8. 
Use importance sampling with $g(x) = \lambda \exp(−\lambda x)$ to calculate the integral $\int_0^1 e^x dx$ of Question 1. Try to find the optimal value of $\lambda$ by calculating the variance of $h(X)f(X)/g(X)$ and verify by simulation. Note that importance sampling with the exponential distribution will not reduce the variance.

**Solution:**

Here we have $g(x) = \lambda \exp(−\lambda x)$, $h(x) = e^x$ and $f(x)=1$.

In [441]:
def h(x):
    return np.exp(x)

def importance_sampling_integral(lambda_, n):
    Y = np.random.exponential(scale=1/lambda_, size=n)
    Y = Y[Y <= 1]  # Truncate to [0, 1]

    w = h(Y) / (lambda_ * np.exp(-lambda_ * Y))
    
    theta = np.mean(w)
    var = np.var(w)
    
    return theta, var

n = 10_000
# A list of lambda
lambdas = np.linspace(1, 5, 50)

thetas = []
vars = []

for lambda_ in lambdas:
    theta, var = importance_sampling_integral(lambda_, n)
    thetas.append(theta)
    vars.append(var)

# Find the optimal lambda, that is the one with the minimum variance
optimal_lambda = lambdas[np.argmin(vars)]

# Print the results
print(f"Optimal lambda: {optimal_lambda}")
print(f"Minimum variance: {min(vars)}")

print(f"Estimate of integral: {np.mean(thetas)}")

Optimal lambda: 1.163265306122449
Minimum variance: 2.6310664429416613
Estimate of integral: 1.9260175659027263


# 9.
 For the Pareto case derive the IS estimator for the mean using the first moment distribution as sampling distribution. Is the approach meaningful? And could this be done in general? With this insight could you change the choice of $g(x)$ in the previous question (Question 8) such that importance sampling would reduce the variance? You do not need to implement this, as long as you can argue, what should happen.

**Solution:** 

Since we are estimating the mean of a Pareto distribution we have $h(x) = x$, $f(x) = \alpha \frac{x_m^{\alpha}}{x^{\alpha +1}}$, that is the pareto PDF and finally $g(x)=\alpha_g \frac{x_m^{\alpha_g}}{x^{\alpha_g +1}}$ where we want to choose $\alpha_g$ such that $g(x)$ closely approximates $h(x)f(x)$ in shape. And that is what we want in general, to find such a distribution $g(x)$.

With this insight we could change the choice of $g(x)$ in Question 8. To reduce the variance in the integral estimation using importance sampling, you should choose a sampling distribution $g(x)$ that closely matches the shape of the product $h(x)f(x)$. For the integral $\int_0^1 e^x dx$, an optimal $g(x)$ can be a truncated exponential distribution with parameter $\lambda$ that approximates the exponential growth of $e^x$ over the interval $[0,1]$. Adjusting $\lambda$ appropriately will reduce the variance of the importance sampling estimator.

In [435]:
def h(x):
    return x
    
def importance_sampling_pareto(n, alpha, xm, alpha_g):
    # Sample from the g sampling distribution with alpha_g
    Y = xm * (np.random.uniform(size=n) ** (-1/alpha_g))

    f = xm * (Y ** (-1/alpha))
    g = xm * (Y ** (-1/alpha_g))
    
    return np.mean(h(Y) * f/g), np.var(h(Y) * f/g)

# Parameters
n = 10_000
alpha = 2.5
xm = 1.0
alpha_g = 2.5

# Run the importance sampling
estimator, variance = importance_sampling_pareto(n, alpha, xm, alpha_g)

print("True mean of the Pareto distribution:", alpha * xm / (alpha - 1))
print("IS Estimator for the mean:", estimator)
print("Variance of the estimator:", variance)


True mean of the Pareto distribution: 1.6666666666666667
IS Estimator for the mean: 1.6714662045013342
Variance of the estimator: 2.7417029433120286
